# Spark ML - Part One

Before doing anything else, let's create a Spark context.

In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(master='local[*]', appName="Spark course Pair RDDs")

For learning how to do linear regression in Spark we will use the housing dataset (https://archive.ics.uci.edu/ml/datasets/Housing). The dataset contains mean values of owner-occupied homes in the suburbs of Boston and 13 features that can be used to predict home values. The data contains the following columns:
1. CRIM      per capita crime rate by town
2. ZN        proportion of residential land zoned for lots over 25,000 sq.ft.
3. INDUS     proportion of non-retail business acres per town
4. CHAS      Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
5. NOX       nitric oxides concentration (parts per 10 million)
6. RM        average number of rooms per dwelling
7. AGE       proportion of owner-occupied units built prior to 1940
8. DIS       weighted distances to five Boston employment centres
9. RAD       index of accessibility to radial highways
10. TAX      full-value property-tax rate per \$10,000
11. PTRATIO  pupil-teacher ratio by town
12. B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
13. LSTAT    % lower status of the population
14. MEDV Median value of owner-occupied homes in $1000's

Load the data using this snippet:

In [2]:
from pyspark.mllib.linalg import Vectors, Vector
housingLines = sc.textFile("/home/spark/first-edition/ch07/housing.data", 4)
housingVals = housingLines.map(lambda x: Vectors.dense([float(v.strip()) for v in x.split(",")]))

Calculate multivariate statistical summary by creating a `pyspark.mllib.linalg.distributed.RowMatrix` variable called *housingMat* from `housingVals` RDD and then call its `computeColumnSummaryStatistics` method.

In [3]:
from pyspark.mllib.linalg.distributed import RowMatrix
housingMat = RowMatrix(housingVals)
housingStats = housingMat.computeColumnSummaryStatistics()

Use the resulting `MultivariateStatisticalSummary` object to find the minimum, maximum and mean values of each column.

In [4]:
print(housingStats.min())
print(housingStats.max())
print(housingStats.mean())

[6.3200e-03 0.0000e+00 4.6000e-01 0.0000e+00 3.8500e-01 3.5610e+00
 2.9000e+00 1.1296e+00 1.0000e+00 1.8700e+02 1.2600e+01 3.2000e-01
 1.7300e+00 5.0000e+00]
[ 88.9762 100.      27.74     1.       0.871    8.78   100.      12.1265
  24.     711.      22.     396.9     37.97    50.    ]
[3.61352356e+00 1.13636364e+01 1.11367787e+01 6.91699605e-02
 5.54695059e-01 6.28463439e+00 6.85749012e+01 3.79504269e+00
 9.54940711e+00 4.08237154e+02 1.84555336e+01 3.56674032e+02
 1.26530632e+01 2.25328063e+01]


Find the L1 norm (method `normL1`), Euclidian norm (method `normL2`) and variance of each column.

In [5]:
print(housingStats.normL1())
print(housingStats.normL2())
print(housingStats.variance())

[1.82844292e+03 5.75000000e+03 5.63521000e+03 3.50000000e+01
 2.80675700e+02 3.18002500e+03 3.46989000e+04 1.92029160e+03
 4.83200000e+03 2.06568000e+05 9.33850000e+03 1.80477060e+05
 6.40245000e+03 1.14016000e+04]
[2.09691067e+02 5.83120056e+02 2.94152392e+02 5.91607978e+00
 1.27463869e+01 1.42248368e+02 1.66721763e+03 9.76051548e+01
 2.90568408e+02 9.93343526e+03 4.17987954e+02 8.28133627e+03
 3.26746015e+02 5.47381348e+02]
[7.39865782e+01 5.43936814e+02 4.70644425e+01 6.45129730e-02
 1.34276357e-02 4.93670850e-01 7.92358399e+02 4.43401514e+00
 7.58163660e+01 2.84047595e+04 4.68698912e+00 8.33475226e+03
 5.09947595e+01 8.45867236e+01]


Execute the following cell to define the helper method that will serve the purpose of printing out matrices in a more readable format.

In [6]:
def printMat(mat):
    if hasattr(mat, 'rows'):
        vals = mat.rows.collect()
        numrows = mat.numRows()
        numcols = mat.numCols()
    elif hasattr(mat, 'toRowMatrix'):
        rm = mat.toRowMatrix()
        vals = rm.rows.collect()
        numrows, numcols = (rm.numRows(), rm.numCols())
    else:
        numrows, numcols = (housingCovar.numRows, housingCovar.numCols)
        vals = housingCovar.values.reshape((numrows, numcols))
    
    print("   " + "".join([("% 10d" % j) for j in range(numcols)]))
    for i in range(numrows):
        print(("%-6d" % i) + "".join(["% 10.3f" % vals[i][j] for j in range(numcols)]))


## Examining the data

Call the `columnSimilarities` method on the *housingMat* matrix and then print it out using the `printMat` function.

In [7]:
housingColSims = housingMat.columnSimilarities()
printMat(housingColSims)

            0         1         2         3         4         5         6         7         8         9        10        11        12        13
0          0.000     0.000     0.000     0.000     0.000     0.966     0.962     0.780     0.808     0.957     0.977     0.929     0.912     0.873
1          0.000     0.004     0.527     0.052     0.459     0.363     0.482     0.169     0.675     0.563     0.416     0.288     0.544     0.224
2          0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.917     0.771     0.642     0.806     0.588
3          0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.670
4          0.000     0.000     0.122     0.078     0.334     0.467     0.211     0.673     0.135     0.297     0.394     0.464     0.200     0.528
5          0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.939     

The higher the values in the cells of this matrix, the more correlated the two columns are. Negative values indicate opposite correlation. The highest value in the last column (corresponding to the target price) is in the sixth row, which corresponds to the average number of rooms. That is the feature that correlates the most with the price of a house.

Next, examine similarities between columns by computing a covariance matrix using the `computeCovariance` method on the *housingMat* matrix. Print out the resulting matrix as you did previously.

In [8]:
housingCovar = housingMat.computeCovariance()
printMat(housingCovar)

            0         1         2         3         4         5         6         7         8         9        10        11        12        13
0         73.987   -40.216    23.992    -0.122     0.420    -1.325    85.405    -6.877    46.848   844.822     5.399  -302.382    27.986   -30.719
1        -40.216   543.937   -85.413    -0.253    -1.396     5.113  -373.902    32.629   -63.349 -1236.454   -19.777   373.721   -68.783    77.315
2         23.992   -85.413    47.064     0.110     0.607    -1.888   124.514   -10.228    35.550   833.360     5.692  -223.580    29.580   -30.521
3         -0.122    -0.253     0.110     0.065     0.003     0.016     0.619    -0.053    -0.016    -1.523    -0.067     1.131    -0.098     0.409
4          0.420    -1.396     0.607     0.003     0.013    -0.025     2.386    -0.188     0.617    13.046     0.047    -4.021     0.489    -0.455
5         -1.325     5.113    -1.888     0.016    -0.025     0.494    -4.752     0.304    -1.284   -34.583    -0.541     

## Preparing the data

To use the data in Spark, you need to put each example in the dataset in a structure called a `LabeledPoint`, which is used in most of Spark’s machine-learning algorithms. The `LabeledPoint` constructor takes the label (the last element in our dataset) as the first argument, and a `DenseVector`  of features (construct it by calling `Vectors.dense`) as the second argument. Transform the `housingVals` RDD like this now. Call the resulting variable `housingData`. 

(Hint: first convert the Vectors in the RDD to arrays by calling `toArray` and then *index into* and `slice` the arrays.)

In [9]:
from pyspark.mllib.regression import LabeledPoint

def vectorToLPoint(v):
    a = v.toArray()
    return LabeledPoint(a[-1], Vectors.dense(a[0:-1]))

housingData = housingVals.map(vectorToLPoint)

Use the `randomSplit` method to split the `housingData` RDD into two new RDDs, `housingTrain` and `housingValid`, containing 80% and 20% of data, respectively.

In [10]:
sets = housingData.randomSplit([0.8, 0.2])
housingTrain = sets[0]
housingValid = sets[1]

Now create two RDDs containing only `features` from `housingTrain` and `housingValid`.

In [11]:
featuresTrain = housingTrain.map(lambda lp: lp.features)
featuresValid = housingValid.map(lambda lp: lp.features)

Next, you need to scale the data so that all features are in the same range (*feature scaling*) and that their averages are roughly zero (*Mean normalization*). Create an instance of `pyspark.mllib.feature.StandardScaler` and specify that you want to do both standardization techniques. Then `fit` the scaler on the RDD containing features from `housingTrain`.

In [12]:
from pyspark.mllib.feature import StandardScaler
scaler = StandardScaler(True, True).fit(featuresTrain)

Now transform both feature RDDs using the fitted scaler and then `zip` the results with the labels from `housingTrain` and `housingValid` RDDs. Finally, transform those results into `LabeledPoint`s and call the resulting RDDs `trainScaled` and `validScaled`. Cache the two resulting RDDs.

In [13]:
trainScaled = housingTrain.map(lambda lp: lp.label).zip(scaler.transform(featuresTrain)).\
  map(lambda lft: LabeledPoint(lft[0], lft[1])).cache()
validScaled = housingValid.map(lambda lp: lp.label).zip(scaler.transform(featuresValid)).\
  map(lambda lft: LabeledPoint(lft[0], lft[1])).cache()

## Fitting and using a linear regression model

Now you can use the prepared data to train a linear regression model. Create a new `pyspark.mllib.regression.LinearRegressionWithSGD` object and train it on `trainScaled` RDD, with the `intercept` flag set to `True` and with 200 iterations. Save the resulting model as a variable.

In [14]:
from pyspark.mllib.regression import LinearRegressionWithSGD
model = LinearRegressionWithSGD.train(trainScaled, iterations=200, intercept=True)

You can now use the trained model to predict the target values of vectors in the validation set by running predict on every element. Map `validScaled` RDD's elements to tuples containing the predicted value (obtained using the model's `predict` method on the features) and the original label. Important: make sure to convert both values to `float`s.

In [15]:
validPredicts = validScaled.map(lambda x: (float(model.predict(x.features)), float(x.label)))

Calculate the root mean squared error (square root of sum of squares of differences between original and predicted target values) using Spark and Python APIs. 

In [16]:
p, l = validPredicts.first()

In [17]:
import math
math.sqrt(validPredicts.map(lambda pl: pow(pl[0]-pl[1],2)).mean())

4.335340896925252

The average value of the target variables (home prices) is 22.5 so your root mean squared error may seem rather large. But take into account that the variance of home prices is 84.6.

Now use `pyspark.mllib.evaluation.RegressionMetrics` evaluate the performance of your regression model. Instantiate it with the RDD containing predictions and then read `rootMeanSquaredError`, `meanSquaredError`, `meanAbsoluteError`, `r2` and `explainedVariance` fields.

In [18]:
from pyspark.mllib.evaluation import RegressionMetrics
validMetrics = RegressionMetrics(validPredicts)
print(validMetrics.rootMeanSquaredError)
print(validMetrics.meanSquaredError)
print(validMetrics.meanAbsoluteError)
print(validMetrics.r2)
print(validMetrics.explainedVariance)

4.335340896925252
18.795180692552645
3.078082986627338
0.7739983577491686
54.03379290540022


Read model's weights and sort them by their absolute values to see which features are the most important (you can convert weights to a list with the `toArray` method).

In [19]:
sorted([(abs(w), ind) for ind, w in enumerate(model.weights.toArray())])

[(0.2098393251764806, 6),
 (0.24366959965218268, 2),
 (0.5965668065825196, 3),
 (0.7266956603940284, 1),
 (0.7378968231306785, 0),
 (0.7628197291548122, 11),
 (0.9450029000070751, 9),
 (1.5902350676874262, 8),
 (1.7001698887611496, 4),
 (2.0279037242924214, 10),
 (2.6029487124670974, 5),
 (2.9233854674555144, 7),
 (3.8761701176232863, 12)]

Save the model to the `output/linearmodel` folder.

In [ ]:
model.save(sc, "output/linearmodel")

You can read the model like this:

In [21]:
from pyspark.mllib.regression import LinearRegressionModel
model_loaded = LinearRegressionModel.load(sc, "output/linearmodel")

## Tweaking the algorithm

Gradient descent has several hyper-parameters that need to be tuned to obtain the best results. For example, the step-size parameter helps to stabilize the gradient descent algorithm. If it’s too small, the algorithm will take too many small steps to converge. If it’s too large, the algorithm may never converge. The right value depends on the dataset.
It’s similar with the number of iterations. If it’s too large, fitting the model will take too much time. If it’s too small, the algorithm may not reach the minimum.

Spark will iterate until the algorithm converges and will determine the optimal number iterations on its own. But you have to find the optimal value for the step size parameter yourself.

The following function will help you experiment with several combinations of these parameters and find the one with best results:

In [22]:
def iterateLRwSGD(iterArray, stepSizeArray, trainRdd, testRdd):
    for numIter in iterArray:
        for step in stepSizeArray:
            alg = LinearRegressionWithSGD.train(trainRdd, iterations=numIter, step=step, intercept=True)
            rescaledPredicts = trainRdd.map(lambda x: (alg.predict(x.features), x.label))
            validPredicts = testRdd.map(lambda x: (alg.predict(x.features), x.label))
            meanSquared = math.sqrt(rescaledPredicts.map(lambda pl: math.pow(pl[0]-pl[1], 2)).mean())
            meanSquaredValid = math.sqrt(validPredicts.map(lambda pl: math.pow(pl[0]-pl[1], 2)).mean())
            print("%d, %5.3f -> %.4f, %.4f" % (numIter, step, meanSquared, meanSquaredValid))

The function will take several values for the number of iterations and step sizes, and two RDDs with train and test data, and will return the RMSE of training and validation sets for each combination of input parameters.

Try it out on our `trainScaled` and `validScaled` RDDs, with 
600 iterations and with step sizes of 0.05, 0.1, 0.5, 1, 1.5, 2 and 3.

In [23]:
iterateLRwSGD([600], [0.05, 0.1, 0.5, 1, 1.5, 2, 3], trainScaled, validScaled)

600, 0.050 -> 7.2564, 6.4885
600, 0.100 -> 5.4933, 4.7533
600, 0.500 -> 4.8589, 4.3512
600, 1.000 -> 4.8112, 4.3353
600, 1.500 -> 4.7930, 4.3249
600, 2.000 -> 4.7873, 4.3213
600, 3.000 -> 700170309.7761, 653469245.7477


You can see several things from this output. First, the testing RMSE is always greater than the training RMSE (except for some corner cases). That’s to be expected. Furthermore, both errors decline rapidly as step size increases, following some inverse exponential function. That makes sense because for smaller smaller step sizes, there weren’t enough iterations to get to the minimum.

Then the error values flatten out. This also makes sense because there are some limitations to how well you can fit a dataset. For a step size value of 3, the error values explode. This step size value is too large, and the
algorithm misses the minimum. It seems that a step size near 1.0 gives the best results.

## Adding higher-order polynomials

Often, data doesn’t follow a simple linear formula (a straight line in a two-dimensional space) but may be some kind of a curve. Curves can often be described with functions containing higher-order polynomials.

Spark doesn’t offer a method of training a nonlinear regression model that includes higher-order polynomials, such as the preceding hypothesis. Instead, you can employ a little trick and do something that has a similar effect: you can expand your dataset with additional features obtained by multiplying the existing ones.

You can use this method for doing that:

In [24]:
def addHighPols(v):
    v1 = [[x, x*x] for x in v.toArray()]
    return Vectors.dense([x for l in v1 for x in l])

Now use it to transform features from the `housingData` RDD and create a new RDD called `housingHP` containing `LabeledPoint` objects with the transformed features.

In [25]:
housingHP = housingData.map(lambda x: LabeledPoint(x.label, addHighPols(x.features)))

Take the first transformed `LabeledPoint` and see how many features it has.

In [26]:
len(housingHP.first().features)

26

Now split and scale the new dataset as you did previously, cache the resulting RDDs and call them `trainHPScaled` and `validHPScaled`.

In [27]:
setsHP = housingHP.randomSplit([0.8, 0.2])
housingHPTrain = setsHP[0]
housingHPValid = setsHP[1]
featuresHPTrain = housingHPTrain.map(lambda lp: lp.features)
featuresHPValid = housingHPValid.map(lambda lp: lp.features)
scalerHP = StandardScaler(True, True).fit(featuresHPTrain)
trainHPScaled = housingHPTrain.map(lambda lp: lp.label).zip(scalerHP.transform(featuresHPTrain)).\
  map(lambda lft: LabeledPoint(lft[0], lft[1])).cache()
validHPScaled = housingHPValid.map(lambda lp: lp.label).zip(scalerHP.transform(featuresHPValid)).\
  map(lambda lft: LabeledPoint(lft[0], lft[1])).cache()

Call `iterateLRwSGD` on these new RDDs with 600 iterations and these step size values: 0.4, 0.5, 0.6, 0.7, 0.9, 1.0, 1.1, 1.2, 1.3, 1.5.

In [28]:
iterateLRwSGD([600], [0.4, 0.5, 0.6, 0.7, 0.9, 1.0, 1.1, 1.2, 1.3, 1.5], trainHPScaled, validHPScaled)

600, 0.400 -> 4.7596, 4.1820
600, 0.500 -> 4.6618, 4.1159
600, 0.600 -> 4.5731, 4.0599
600, 0.700 -> 4.4938, 4.0125
600, 0.900 -> 4.3642, 3.9419
600, 1.000 -> 4.3130, 3.9169
600, 1.100 -> 4.2679, 3.8966
600, 1.200 -> 4.2378, 3.8901
600, 1.300 -> 64.8313, 55.5298
600, 1.500 -> 52494653.3600, 52545943.4764


As you can see from the results, the RMSE explodes for a step size of 1.3, and you get the best results for a step
size of 1.1 or 1.2. The error values are lower than before. We can conclude that adding higher-order polynomials helped the linear-regression algorithm find a better-performing model.

## Lasso and Ridge regressions

You can add regularization to linear regression training in Spark with Lasso and Ridge regressions. These two methods can help with training such models:

In [29]:
from pyspark.mllib.regression import RidgeRegressionWithSGD
from pyspark.mllib.regression import LassoWithSGD

def iterateRidge(iterNums, stepSizes, regParam, train, test):
    for numIter in iterNums:
        for step in stepSizes:
            model = RidgeRegressionWithSGD.train(train, iterations=numIter, regParam=regParam, step=step, intercept=True)
            rescaledPredicts = train.map(lambda x: (model.predict(x.features), x.label))
            validPredicts = test.map(lambda x: (model.predict(x.features), x.label))
            meanSquared = math.sqrt(rescaledPredicts.map(lambda pl: math.pow(pl[0]-pl[1], 2)).mean())
            meanSquaredValid = math.sqrt(validPredicts.map(lambda pl: math.pow(pl[0]-pl[1], 2)).mean())
            print("%d, %5.3f -> %.4f, %.4f" % (numIter, step, meanSquared, meanSquaredValid))

def iterateLasso(iterNums, stepSizes, regParam, train, test):
    for numIter in iterNums:
        for step in stepSizes:
            modell = LassoWithSGD.train(train, iterations=numIter, step=step, regParam=regParam, intercept=True)
            rescaledPredicts = train.map(lambda x: (modell.predict(x.features), x.label))
            validPredicts = test.map(lambda x: (modell.predict(x.features), x.label))
            meanSquared = math.sqrt(rescaledPredicts.map(lambda pl: math.pow(pl[0]-pl[1], 2)).mean())
            meanSquaredValid = math.sqrt(validPredicts.map(lambda pl: math.pow(pl[0]-pl[1], 2)).mean())
            print("%d, %5.3f -> %.4f, %.4f" % (numIter, step, meanSquared, meanSquaredValid))
            print("\tweights: "+str(model.weights))

Both of these methods add another parameter (the third one): the regularization parameter. Try them out with the same RDDs, with 600 iterations, the same step size and the regularization parameter of 0.01.

In [30]:
iterateRidge([600], [1.2], 0.01, trainHPScaled, validHPScaled)
iterateLasso([600], [1.2], 0.01, trainHPScaled, validHPScaled)

600, 1.200 -> 4.3440, 3.9373
600, 1.200 -> 4.2369, 3.8730
	weights: [-0.7378968231306785,0.7266956603940284,-0.24366959965218268,0.5965668065825196,-1.7001698887611496,2.6029487124670974,-0.2098393251764806,-2.9233854674555144,1.5902350676874262,-0.9450029000070751,-2.0279037242924214,0.7628197291548122,-3.8761701176232863]


As you can see, for this dataset regularization doesn't help much. It would be different if you had a problem with overfitting.

## Performance optimization

If your dataset is very large so that it doesn't fit into memory or you want to speed up the training, you can use *mini-batch stochastic gradient descent* optimizer. Instead of going through all the data points in each iteration, it will consider only some batch of data. It has more trouble converging, but is much more resource efficient.

This function will allow you to test mini-batch SGD with several combinations of iterations, step sizes and mini-batch fractions. 

In [31]:
def iterateLRwSGDBatch(iterNums, stepSizes, fractions, train, test):
    for numIter in iterNums:
        for step in stepSizes:
            for mbf in fractions:
                model = LinearRegressionWithSGD.train(train, iterations=numIter, step=step, 
                                                      miniBatchFraction=mbf, intercept=True)
                rescaledPredicts = train.map(lambda x: (model.predict(x.features), x.label))
                validPredicts = test.map(lambda x: (model.predict(x.features), x.label))
                meanSquared = math.sqrt(rescaledPredicts.map(lambda pl: math.pow(pl[0]-pl[1], 2)).mean())
                meanSquaredValid = math.sqrt(validPredicts.map(lambda pl: math.pow(pl[0]-pl[1], 2)).mean())
                print("%d, %5.3f %5.3f -> %.4f, %.4f" % (numIter, step, mbf, meanSquared, meanSquaredValid))

Test it now using `trainHPScaled` and `validHPScaled` RDDs, with 600 iterations, step sizes of 0.1, 0.2, 0.3, 0.4, 0.5, and mini-batch fractions of 0.01, 0.1, 0.3, 0.5, 0.8.

In [32]:
iterateLRwSGDBatch([600], [0.1, 0.2, 0.3, 0.4, 0.5], [0.01, 0.1, 0.3, 0.5, 0.8], trainHPScaled, validHPScaled)

600, 0.100 0.010 -> 6.2449, 5.6129
600, 0.100 0.100 -> 5.3266, 4.7609
600, 0.100 0.300 -> 5.4031, 4.8421
600, 0.100 0.500 -> 5.3769, 4.8068
600, 0.100 0.800 -> 5.4362, 4.9046
600, 0.200 0.010 -> 7.0249, 6.3063
600, 0.200 0.100 -> 4.7465, 4.1597
600, 0.200 0.300 -> 4.8229, 4.2129
600, 0.200 0.500 -> 4.8854, 4.2282
600, 0.200 0.800 -> 4.9318, 4.2925
600, 0.300 0.010 -> 5.5822, 5.0739
600, 0.300 0.100 -> 4.6132, 4.0256
600, 0.300 0.300 -> 4.6698, 4.1001
600, 0.300 0.500 -> 4.7343, 4.1451
600, 0.300 0.800 -> 4.8012, 4.1714
600, 0.400 0.010 -> 18.7670, 18.5032
600, 0.400 0.100 -> 4.5220, 3.9502
600, 0.400 0.300 -> 4.5236, 4.0198
600, 0.400 0.500 -> 4.6039, 4.1153
600, 0.400 0.800 -> 4.6716, 4.1054
600, 0.500 0.010 -> 474.3977, 454.9575
600, 0.500 0.100 -> 5.2657, 4.5342
600, 0.500 0.300 -> 4.5402, 4.1871
600, 0.500 0.500 -> 4.6386, 4.1886
600, 0.500 0.800 -> 4.5032, 4.0003


As you can see, the mini-batch fraction significantly influences the best performance you can get, but that is still worse than the classic SGD. If you have lots of data, that difference might be worth it.

Finally, LBFGS algorithm can computationaly outperform all of the algorithms we've seen so far. LBFGS is a limited-memory approximation of the Broyden-Fletcher-Goldfarb-Shanno (BFGS) algorithm for minimizing multidimensional functions. 

Unfortunatelly however, LBFGS algorithm for linear regression is not available in Python.